## Pour éxécuter ce notebook : 

- télécharger les données 3d ici https://we.tl/t-g9Y9v6dROU et les placer dans le répertoire `3d_data/`
- créer un le répertoire `GRM_data/`

## Ce notebook sert à créer le modèle graphique suivant : 

Lien de téléchargement : 

**Format de l'entrée :** 
- 1ère ligne : $n, m$ deux entiers. $n$ le nombre points, $m$ nombre d'arrêtes.
- $n$ lignes suivantes : matrice $P \in [0, 1]^{n \times 6}$. $P_{ic} =$ probabilité que le points $i$ soit de label $c$ (d'après le classifier).
- $m$ lignes suivantes : arrêtes au format $i, j, d_{ij} \in \{0, ..., n-1\} \times \{0, ..., n-1\} \times \mathbb R_+$ 

On a $n \approx 10^6$ et $m \approx 5 n$

**Format de la sorties :**
- n lignes : $x \in \{1, ..., 6\}^n$ résulats de la minimisation de l'énergie $E(x)$ avec BP ou TRW. 

**Energie $E(x)$:**

$E(x) = \sum_{i=1}^n f(P_{i, x_i}) +  \sum_{(i, j) \in \mathcal E} g(d_{ij}) \mathbb 1_{x_i \neq x_j} $

Avec $f : [0, 1] \rightarrow \mathbb R$ décroissante et $g : \mathbb R_+ \rightarrow \mathbb R_+$ décroissante. Pour commencer on peut prendre : 

$E(x) = \sum_{i=1}^n - P_{i, x_i} +  \sum_{(i, j) \in \mathcal E} \alpha \frac{1}{d_{ij}} \mathbb 1_{x_i \neq x_j} $ avec $\alpha \in \mathbb R_+$ à régler 

Cette modélisation vient exprimer le fait suivant : "deux points qui sont proches ont une forte chance de partager le même label". Le but de cette modélisation/minimisation est de trouver des labels $x$ meilleurs que ceux du classifier (qui sont les $(\arg\max_c P_{ic})_i$).

**Score** :

Le score est calculer par rapports aux vrais labels $x^{true} \in \{0, 1, ..., 6\}^n$ (le label $0$ représente les "unclassifed". Les points "unclassifed" n'interviennent pas dans le score.) 

$\text{Score}(x^{true}, x) = \frac 1 6 \sum_{c = 1}^6 \text{IoU}(x^{true}, x, c)$ où $\text{IoU}$ signifie Intersection over Union : $$\text{IoU}(x^{true}, x, c) = \frac{ \{i, x^{true}_i = c\} \cap  \{i, x_i = c\} }{\{i, x^{true}_i = c\} \cup  \{i, x_i = c \text{ et } x^{true}_i \neq 0\}}$$


Le classifier fait entre $0.35$ et $0.5$.

In [2]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import lightgbm as lgb
from multiprocessing import Pool
from pathlib import Path

import pickle
from cloudpoints import load_points
from datasets import CloudDataset, MultiDataset, label_names
import pandas as pd

In [4]:
data_path = Path("3d_data/")

paris, paris_labels = load_points(data_path / "MiniParis1.ply")
lille1, lille1_labels = load_points(data_path / "MiniLille1.ply")
lille2, lille2_labels = load_points(data_path / "MiniLille2.ply")
dijon = load_points(data_path / "MiniDijon9.ply")
paris_wh = paris[:, 1] <= 20
paris1, paris1_label = paris[paris_wh], paris_labels[paris_wh]
paris2, paris2_label = paris[~paris_wh], paris_labels[~paris_wh]

datasets = [
    CloudDataset("paris1", paris1, paris1_label),
    CloudDataset("paris2", paris2, paris2_label),
    CloudDataset("lille1", lille1, lille1_labels),
    CloudDataset("lille2", lille2, lille2_labels),
    CloudDataset("dijon", dijon)
]
datasets

[CloudDataset<paris1, 1896865 points, labeled>,
 CloudDataset<paris2, 2262453 points, labeled>,
 CloudDataset<lille1, 1901853 points, labeled>,
 CloudDataset<lille2, 2500428 points, labeled>,
 CloudDataset<dijon, 3079187 points, unlabeled>]

In [5]:
for dataset in datasets:
    dataset.compute_neighborhoods([(300, 0.25), (150, 0.15), (50, 0.1)])

In [7]:
def compute_4(an, eigvals, eigs):
    verticality = 2/np.pi * np.arcsin(np.abs(eigs[:, -1, -1]))
    linearity = 1 - eigvals[:, 1] / np.minimum(eigvals[:, 0], 1e-8)
    planarity = (eigvals[:, 1] - eigvals[:, 2]) / np.minimum(eigvals[:, 0], 1e-8)
    sphericity = eigvals[:, 2] / np.minimum(eigvals[:, 0], 1e-8)
    return np.vstack((verticality, linearity, planarity, sphericity)).T

def raw_eigenvalues(an, eigvals, eigs):
    return eigvals

def raw_eigenvector(an, eigvals, eigs):
    return eigs.reshape(-1, 9)

def density(ans, eigvals, eigs):
    return np.array([ns.size for ns in ans], dtype=float).reshape(-1, 1)

feature_functions = [compute_4, raw_eigenvalues, raw_eigenvector, density]

for dataset in datasets:
    dataset.compute_features(feature_functions)

/home/infres/alacote/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until
/home/infres/alacote/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in true_divide
  after removing the cwd from sys.path.
/home/infres/alacote/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.
/home/infres/alacote/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in true_divide
  """


In [8]:
test_idx = 1
train_multidataset = MultiDataset(datasets[:-1], test_idx)
X_train, X_test, y_train, labels_test = train_multidataset.train_test_split()
np.bincount(labels_test)

array([199742, 790686, 501958,  14467,  10567,  37790, 707243])

In [9]:
num_round = 50
param = {'num_leaves': 31, 'max_depth': -1, 'objective': 'multiclass', 'num_class': 6, 'max_bin': 30}

train_data = lgb.Dataset(X_train, label=y_train)
bst = lgb.train(param, train_data, num_round)
y_pred = np.concatenate((np.zeros((X_test.shape[0], 1)), bst.predict(X_test)), axis=1) 
labels_pred = np.argmax(y_pred[:, 1:], axis=1) + 1

In [10]:
def print_results(labels_test, labels_pred):
    precisions = []
    recalls = []
    IoUs = []
    for c in range(1, 7):
        if(np.sum(labels_test == c) == 0):
            precisions.append(1)
            recalls.append(1)
            IoUs.append(1)
        else:
            precisions.append(np.mean(labels_test[labels_pred == c] == c))
            recalls.append(np.mean(labels_pred[labels_test == c] == c))
            IoUs.append(np.sum((labels_pred == c) & (labels_test == c)) / \
                        np.sum(((labels_pred == c) & (labels_test != 0)) | (labels_test == c)))
        

    return pd.DataFrame({"precision": precisions, "recall": recalls, "IoU": IoUs}, 
                        index=[str(i) + " - " + label_names[i] for i in range(1, 7)])

res = print_results(labels_test, labels_pred)
print("Score:", res["IoU"].mean())
res

Score: 0.4826614824699731


,precision,recall,IoU
1 - Ground,0.944321,0.968637,0.928423
2 - Building,0.634608,0.873523,0.731533
3 - Poles,0.497966,0.253888,0.209742
4 - Pedestrians,0.284959,0.127472,0.104072
5 - Cars,0.508233,0.115983,0.111357
6 - Vegetation,0.858031,0.897464,0.810842


Write the Graphical Model as described above (and the corresponding labels) : 

In [25]:
class Neighborhoods:
    
    def __init__(self, points, neigh_idxs=None, distances=None, k=None):
        self.points = points
        self.n = points.shape[0]
        if(k is None):
            self.neigh_idxs = neigh_idxs
            self.distances = distances
        else:
            kdt = KDTree(points)
            self.distances, self.neigh_idxs = kdt.query(points, k=k)
    
    def restrict(self, k=10000, radius=1e9, inplace=False):
        whs = [ds < radius for ds in self.distances]
        for wh in whs:
            wh[k:] = False
        res_neigh_idxs = []
        res_distances = []
        for wh, ns, ds in zip(whs, self.neigh_idxs, self.distances):
            res_neigh_idxs.append(ns[wh])
            res_distances.append(ds[wh])
        if(inplace):
            self.neigh_idxs = res_neigh_idxs
            self.distances = res_distances
            return self
        else:
            return Neighborhoods(self.points, res_neigh_idxs, res_distances)
    
    def compute_local_PCAs(self):
        self.eigenvalues = np.zeros((self.n, 3))
        self.eigenvectors = np.zeros((self.n, 3, 3))
        for i, ns in enumerate(self.neigh_idxs):
            neighs = self.points[ns, :]
            eigvals, eigvects = local_PCA(neighs)
            self.eigenvalues[i, :] = eigvals
            self.eigenvectors[i, :] = eigvects
            
    def get_features(self, feature_functions):
        features = []
        neighs = [self.points[ns, :] for ns in self.neigh_idxs]
        for func in feature_functions:
            f = func(neighs, self.eigenvalues, self.eigenvectors)
            if(f.ndim == 1):
                f = f.reshape(-1, 1)
            features.append(f)
        return np.concatenate(features, axis=1)
    
    def get_edges(self):
        m = np.sum([ns.size for ns in self.neigh_idxs])
        edges = np.zeros((m, 2), dtype=int)
        j = 0
        for i, ns in enumerate(self.neigh_idxs):
            edges[j:j + ns.size, 0] = i
            edges[j:j + ns.size, 1] = ns
            j += ns.size
        edges = edges[:j, :]
        edges = edges[edges[:, 0] != edges[:, 1], :]
        edges = drop_duplicated_rows(edges)
        distances = np.linalg.norm(self.points[edges[:, 0], :] - self.points[edges[:, 1], :], axis=1)
        return edges, distances
    
from cloudpoints import drop_duplicated_rows

In [32]:
ds = datasets[0]
nb = Neighborhoods(ds.points, ds.neighborhoods[-1].neigh_idxs, ds.neighborhoods[-1].distances)
edges, _ = nb.restrict(k=12, radius=0.085).get_edges()

In [34]:
nb = ds.neighborhoods[0]

In [36]:
for dataset_idx in [0, 1, 2, 3]:
    dataset = datasets[dataset_idx]
    new_nbs = []
    for nb in dataset.neighborhoods:
        new_nb = Neighborhoods(nb.points, nb.neigh_idxs, nb.distances)
        new_nb.eigenvalues = nb.eigenvalues
        new_nb.eigenvectors = nb.eigenvectors
        new_nbs.append(new_nb)
    dataset.neighborhoods = new_nbs

In [37]:
num_round = 50
param = {'num_leaves': 31, 'max_depth': -1, 'objective': 'multiclass', 'num_class': 6, 'max_bin': 30}

for dataset_idx in [0, 1, 2, 3]:
    train_multidataset = MultiDataset(datasets[:-1], dataset_idx)
    X_train, X_test, y_train, labels_test = train_multidataset.train_test_split()
    train_data = lgb.Dataset(X_train, label=y_train)
    bst = lgb.train(param, train_data, num_round)
    y_pred = np.concatenate((np.zeros((X_test.shape[0], 1)), bst.predict(X_test)), axis=1) 
    labels_pred = np.argmax(y_pred[:, 1:], axis=1) + 1
    
    res = print_results(labels_test, labels_pred)
    print("avg IoU:", res["IoU"].mean())
    worst_class = res["IoU"].idxmin()
    print("Worst class :", worst_class, "| IoU:", res.loc[worst_class, "IoU"])
    
    datasets[dataset_idx].write_GRM(y_pred[:, 1:], Path("GRM_data"), grm_k=11, grm_radius=0.085)

avg IoU: 0.36206471884456626
Worst class : 5 - Cars | IoU: 0.014310664626798987
avg IoU: 0.4826614824699731
Worst class : 4 - Pedestrians | IoU: 0.10407169898786989
avg IoU: 0.5339834812471024
Worst class : 3 - Poles | IoU: 0.03163660598991961
avg IoU: 0.5866858213235947
Worst class : 5 - Cars | IoU: 0.11176871622088945
